In [1]:

import os
model_save_path = "/home/sys1/Dong/A_Stage_Two/ABSA-QUAD/data/clean/uausal/fasttext"  #名字
train_data_path = "/home/sys1/Dong/A_Stage_Two/ABSA-QUAD/data/clean/uausal/train1.txt"
validate_data_path = "/home/sys1/Dong/A_Stage_Two/ABSA-QUAD/data/clean/uausal/dev1.txt"
test_data_path = "/home/sys1/Dong/A_Stage_Two/ABSA-QUAD/data/clean/uausal/test1.txt"
test_data_predict_out_path = "/home/sys1/Dong/A_Stage_Two/ABSA-QUAD/data/clean/uausal/" 

In [2]:
import pandas as pd
import jieba
import json

label2id = {'happy': 0, 'angry': 1, 'sad': 2, 'fear': 3, 'surprise': 4, 'neutral': 5}
def load_data(filename):
    D = []
    f = json.load(open(filename))    
    for l in f:
        label, content = label2id[l['label']],l['content']
        D.append((label, content))
    return D

# 分词
def seg_words(contents):
    contents_segs = list()
    for content in contents:
        segs = jieba.lcut(content[1])
        # print(segs)
        contents_segs.append(" ".join(segs))

    return contents_segs
# content_train = load_data(train_data_path)
# s = seg_words(content_train)
# # print(s[1])

In [4]:
import argparse
import logging
import os

import numpy as np
from sklearn.metrics import f1_score,accuracy_score
from skift import FirstColFtClassifier
import joblib


logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] <%(processName)s> (%(threadName)s) %(message)s')
logger = logging.getLogger(__name__)

if __name__ == '__main__':

    model_name = 'fasttext_model.pkl'
    learning_rate = 1.0
    epoch = 10
    word_ngrams = 1
    min_count = 1

    # load train data
    logger.info("start load load")
    content_train = load_data(train_data_path)
    

    labels_train = []
    for l in content_train:
        labels_train .append(l[0])

    logger.info("start seg train data")
    content_train = seg_words(content_train)
    logger.info("complete seg train data")

    logger.info("prepare train format")
    train_data_format = np.asarray([content_train]).T
    logger.info("complete formate train data")



    # model train
    logger.info("start train model")
    
    
    sk_clf = FirstColFtClassifier(lr=learning_rate, epoch=epoch,
                                    wordNgrams=word_ngrams,
                                    minCount=min_count, verbose=2)
    sk_clf.fit(train_data_format, labels_train)
    classifier_dict = sk_clf

    logger.info("complete train model")
    logger.info("start save model")
    model_path = model_save_path
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    joblib.dump(classifier_dict, model_path + model_name)
    logger.info("complete svae model")

    # validata model
    content_validata = load_data(validate_data_path)
    labels_validate = []
    for l in content_validata:
        labels_validate .append(l[0])

    logger.info("start seg validata data")
    content_validata = seg_words(content_validata)
    logger.info("complet seg validata data")

    logger.info("prepare valid format")
    validata_data_format = np.asarray([content_validata]).T
    logger.info("complete formate train data")

    logger.info("start compute f1 score for validata model")
    
    true_label = np.asarray(labels_validate)
    classifier = classifier_dict
    pred_label = classifier.predict(validata_data_format).astype(int)
    f1_score1 = f1_score(true_label, pred_label,average='macro')
    acc = accuracy_score(true_label, pred_label)

    logger.info("acc: %s\n" % acc)
    logger.info("f1_score: %s" % f1_score1)

2023-03-27 19:49:12,561 [INFO] <MainProcess> (MainThread) start load load
2023-03-27 19:49:12,653 [INFO] <MainProcess> (MainThread) start seg train data
Building prefix dict from the default dictionary ...
2023-03-27 19:49:12,654 [DEBUG] <MainProcess> (MainThread) Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
2023-03-27 19:49:12,656 [DEBUG] <MainProcess> (MainThread) Loading model from cache /tmp/jieba.cache
Loading model cost 0.652 seconds.
2023-03-27 19:49:13,308 [DEBUG] <MainProcess> (MainThread) Loading model cost 0.652 seconds.
Prefix dict has been built successfully.
2023-03-27 19:49:13,311 [DEBUG] <MainProcess> (MainThread) Prefix dict has been built successfully.
2023-03-27 19:49:18,418 [INFO] <MainProcess> (MainThread) complete seg train data
2023-03-27 19:49:18,421 [INFO] <MainProcess> (MainThread) prepare train format
2023-03-27 19:49:18,449 [INFO] <MainProcess> (MainThread) complete formate train data
2023-03-27 19:49:18,450 

In [7]:
#predict
import argparse
import logging

import numpy as np

import joblib


logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] <%(processName)s> (%(threadName)s) %(message)s')
logger = logging.getLogger(__name__)


if __name__ == '__main__':


    model_name = 'fasttext_model.pkl'

    # load data
    logger.info("start load load")
    content_test = load_data(test_data_path)
    labels_test = []
    for l in content_test:
        labels_test.append(l[0])
    # load model
    logger.info("start load model")
    classifier_dict = joblib.load(model_path + model_name)

    logger.info("start seg train data")
    content_test = seg_words(content_test)
    logger.info("complete seg train data")

    logger.info("prepare predict data format")
    test_data_format = np.asarray([content_test]).T
    logger.info("complete prepare predict formate data")

    true_label = np.asarray(labels_test)
    # model predict
    logger.info("start predict test data")
    test_pre = classifier_dict.predict(test_data_format).astype(int)
    
    f1_score1 = f1_score(true_label, test_pre,average='macro')
    acc = accuracy_score(true_label, test_pre)
    
    logger.info("complete predict test data")
    logger.info("acc: %s\n" % acc)
    logger.info("f1_score: %s" % f1_score1)

2023-03-27 19:51:13,419 [INFO] <MainProcess> (MainThread) start load load
2023-03-27 19:51:13,440 [INFO] <MainProcess> (MainThread) start load model
2023-03-27 19:51:13,534 [INFO] <MainProcess> (MainThread) start seg train data
2023-03-27 19:51:14,476 [INFO] <MainProcess> (MainThread) complete seg train data
2023-03-27 19:51:14,478 [INFO] <MainProcess> (MainThread) prepare predict data format
2023-03-27 19:51:14,482 [INFO] <MainProcess> (MainThread) complete prepare predict formate data
2023-03-27 19:51:14,484 [INFO] <MainProcess> (MainThread) start predict test data
2023-03-27 19:51:14,560 [INFO] <MainProcess> (MainThread) complete predict test data
2023-03-27 19:51:14,561 [INFO] <MainProcess> (MainThread) acc: 0.6918

2023-03-27 19:51:14,562 [INFO] <MainProcess> (MainThread) f1_score: 0.6587498749380203
